# 기본 설정



In [28]:
import pandas as pd
import numpy as np
pd.options.display.max_rows = 100
pd.options.display.max_columns = 100

# 지도시각화
import folium
from folium import plugins
import html
import json
import os
import requests
import ipywidgets
from IPython.display import Image
from ipywidgets import interact

In [2]:
df = pd.read_csv('데이터 모음/대전광역시 상점정보들.csv')

In [3]:
df.head()

,상가업소번호,상호명,상권업종대분류명,상권업종중분류명,상권업종소분류명,시군구명,법정동명,지번주소,도로명주소,층정보,경도,위도
0,20052813,한미푸드,소매,음/식료품소매,식자재판매,중구,대흥동,대전광역시 중구 대흥동 524-4,대전광역시 중구 대흥로28번길 47,1.0,127.419096,36.323250
1,20052915,산봉냉면,음식,한식,냉면집,서구,둔산동,대전광역시 서구 둔산동 1036,대전광역시 서구 대덕대로 211,2.0,127.378173,36.351866
2,20053087,돌다리식당,음식,한식,한식/백반/한정식,중구,석교동,대전광역시 중구 석교동 54-2,대전광역시 중구 대종로 234-17,1.0,127.442208,36.310580
3,20053876,또래오래,음식,닭/오리요리,후라이드/양념치킨,중구,태평동,대전광역시 중구 태평동 327-32,대전광역시 중구 수침로55번길 37,1.0,127.398644,36.325212
4,20055302,무지개횟집,음식,일식/수산물,횟집,서구,정림동,대전광역시 서구 정림동 610,대전광역시 서구 정림동로7번길 22,1.0,127.366426,36.305534


# 약국만 뽑기

In [5]:
df.columns

Index(['상가업소번호', '상호명', '상권업종대분류명', '상권업종중분류명', '상권업종소분류명', '시군구명', '법정동명',
       '지번주소', '도로명주소', '층정보', '경도', '위도'],
      dtype='object')

In [6]:
df['상권업종대분류명'].unique()

array(['소매', '음식', '생활서비스', '학문/교육', '부동산', '의료', '관광/여가/오락', '숙박', '스포츠'],
      dtype=object)

In [8]:
medi = df[df['상권업종대분류명'] == '의료']

In [9]:
medi['상권업종중분류명'].unique()

array(['병원', '유사의료업', '약국/한약방', '수의업', '의료관련서비스업'], dtype=object)

In [16]:
hosp = medi[medi['상권업종중분류명'] == '병원']
temp = medi[medi['상권업종중분류명'] == '약국/한약방']
phar = temp[temp['상권업종소분류명'] == '약국']

병원이랑 약국 정보 뽑았음!

In [18]:
hosp.head()

,상가업소번호,상호명,상권업종대분류명,상권업종중분류명,상권업종소분류명,시군구명,법정동명,지번주소,도로명주소,층정보,경도,위도
81,21718264,한민내과의원,의료,병원,내과/외과,중구,유천동,대전광역시 중구 유천동 210-8,대전광역시 중구 계백로 1592,2.0,127.397196,36.318004
85,21907594,민들레의원,의료,병원,기타병원,대덕구,법동,대전광역시 대덕구 법동 282-7,대전광역시 대덕구 계족로663번길 22,NaN,127.427525,36.370741
130,22628544,한밭연세의원,의료,병원,기타병원,중구,오류동,대전광역시 중구 오류동 152-9,대전광역시 중구 계백로 1727,3.0,127.411040,36.322336
156,23070824,관저성모의원,의료,병원,일반병원,서구,관저동,대전광역시 서구 관저동 984-2,대전광역시 서구 관저북로 78,2.0,127.337316,36.308603
237,25576258,스타힐성형피부과,의료,병원,피부과,서구,월평동,대전광역시 서구 월평동 248,대전광역시 서구 월평북로 88,5.0,127.377924,36.362076


약국 전처리

In [21]:
phar.reset_index(inplace=True)

In [25]:
phar = phar.drop(columns=['index'])

In [26]:
phar

,상가업소번호,상호명,상권업종대분류명,상권업종중분류명,상권업종소분류명,시군구명,법정동명,지번주소,도로명주소,층정보,경도,위도
0,21441109,관저태평양약국,의료,약국/한약방,약국,유성구,대정동,대전광역시 유성구 대정동 312-6,대전광역시 유성구 대정로28번안길 83,1.0,127.321083,36.313414
1,21841763,보왕약국,의료,약국/한약방,약국,서구,월평동,대전광역시 서구 월평동 1168,대전광역시 서구 월평서로 7,NaN,127.358010,36.354220
2,21834969,조일약국,의료,약국/한약방,약국,동구,삼성동,대전광역시 동구 삼성동 372-7,대전광역시 동구 현암로 18,NaN,127.421102,36.342754
3,22007905,동아약국,의료,약국/한약방,약국,서구,관저동,대전광역시 서구 관저동 1340,대전광역시 서구 관저로 157,NaN,127.339777,36.299306
4,21971486,건강한약국,의료,약국/한약방,약국,유성구,송강동,대전광역시 유성구 송강동 195-13,대전광역시 유성구 구즉로 48,1.0,127.384818,36.433355
...,...,...,...,...,...,...,...,...,...,...,...,...
599,16196725,온누리약국,의료,약국/한약방,약국,서구,둔산동,대전광역시 서구 둔산동 1507,대전광역시 서구 문예로 67,NaN,127.389865,36.352728
600,16269430,메디팜,의료,약국/한약방,약국,중구,선화동,대전광역시 중구 선화동 141-2,대전광역시 중구 대종로549번길 34,1,127.420667,36.330765
601,16248227,해오름약국,의료,약국/한약방,약국,동구,대동,대전광역시 동구 대동 200-18,대전광역시 동구 계족로 116,1,127.446128,36.324703
602,16269971,강산약국,의료,약국/한약방,약국,동구,효동,대전광역시 동구 효동 253-18,대전광역시 동구 보문로 4,NaN,127.440091,36.315374


# 약국 뿌려보기

In [40]:
phar

,상가업소번호,상호명,상권업종대분류명,상권업종중분류명,상권업종소분류명,시군구명,법정동명,지번주소,도로명주소,층정보,경도,위도
0,21441109,관저태평양약국,의료,약국/한약방,약국,유성구,대정동,대전광역시 유성구 대정동 312-6,대전광역시 유성구 대정로28번안길 83,1.0,127.321083,36.313414
1,21841763,보왕약국,의료,약국/한약방,약국,서구,월평동,대전광역시 서구 월평동 1168,대전광역시 서구 월평서로 7,NaN,127.358010,36.354220
2,21834969,조일약국,의료,약국/한약방,약국,동구,삼성동,대전광역시 동구 삼성동 372-7,대전광역시 동구 현암로 18,NaN,127.421102,36.342754
3,22007905,동아약국,의료,약국/한약방,약국,서구,관저동,대전광역시 서구 관저동 1340,대전광역시 서구 관저로 157,NaN,127.339777,36.299306
4,21971486,건강한약국,의료,약국/한약방,약국,유성구,송강동,대전광역시 유성구 송강동 195-13,대전광역시 유성구 구즉로 48,1.0,127.384818,36.433355
...,...,...,...,...,...,...,...,...,...,...,...,...
599,16196725,온누리약국,의료,약국/한약방,약국,서구,둔산동,대전광역시 서구 둔산동 1507,대전광역시 서구 문예로 67,NaN,127.389865,36.352728
600,16269430,메디팜,의료,약국/한약방,약국,중구,선화동,대전광역시 중구 선화동 141-2,대전광역시 중구 대종로549번길 34,1,127.420667,36.330765
601,16248227,해오름약국,의료,약국/한약방,약국,동구,대동,대전광역시 동구 대동 200-18,대전광역시 동구 계족로 116,1,127.446128,36.324703
602,16269971,강산약국,의료,약국/한약방,약국,동구,효동,대전광역시 동구 효동 253-18,대전광역시 동구 보문로 4,NaN,127.440091,36.315374


In [52]:
map_daejeon = folium.Map(location=[36.34722222, 127.39027778], zoom_start=14.4)

# 구 이름으로 나누기
서구 = phar[phar['시군구명']=='서구']
중구 = phar[phar['시군구명']=='중구']
유성구 = phar[phar['시군구명']=='유성구']
동구 = phar[phar['시군구명']=='동구']
대덕구 = phar[phar['시군구명']=='대덕구']

# 서구
for shop in range(서구.shape[0]):
    folium.Marker([서구['위도'].iloc[shop], 서구['경도'].iloc[shop]], popup=서구['상호명'].iloc[shop], icon=folium.Icon(icon='info-sign', color = "blue")).add_to(map_daejeon)
# 중구
for shop in range(중구.shape[0]):
    folium.Marker([중구['위도'].iloc[shop], 중구['경도'].iloc[shop]], popup=중구['상호명'].iloc[shop], icon=folium.Icon(icon='info-sign', color = "red")).add_to(map_daejeon)
# 유성구
for shop in range(유성구.shape[0]):
    folium.Marker([유성구['위도'].iloc[shop], 유성구['경도'].iloc[shop]], popup=유성구['상호명'].iloc[shop], icon=folium.Icon(icon='info-sign', color = "green")).add_to(map_daejeon)
# 동구
for shop in range(동구.shape[0]):
    folium.Marker([동구['위도'].iloc[shop], 동구['경도'].iloc[shop]], popup=동구['상호명'].iloc[shop], icon=folium.Icon(icon='info-sign', color = "yellow")).add_to(map_daejeon)
# 대덕구
for shop in range(대덕구.shape[0]):
    folium.Marker([대덕구['위도'].iloc[shop], 대덕구['경도'].iloc[shop]], popup=대덕구['상호명'].iloc[shop], icon=folium.Icon(icon='info-sign', color = "black")).add_to(map_daejeon)  

<ipython-input-52-d99650ea1eda>:21: UserWarning: color argument of Icon should be one of: {'darkred', 'darkgreen', 'darkblue', 'gray', 'cadetblue', 'lightgreen', 'lightgray', 'white', 'black', 'lightred', 'pink', 'green', 'lightblue', 'purple', 'darkpurple', 'beige', 'blue', 'red', 'orange'}.
  folium.Marker([동구['위도'].iloc[shop], 동구['경도'].iloc[shop]], popup=동구['상호명'].iloc[shop], icon=folium.Icon(icon='info-sign', color = "yellow")).add_to(map_daejeon)


In [54]:
map_daejeon.save('대전광역시 약국.html')

In [53]:
map_daejeon

# 그리드 깔기

In [55]:
## 거리 구하는 함수
import numbers
import math

class GeoUtil:
    """
    Geographical Utils
    """
    @staticmethod
    def degree2radius(degree):
        return degree * (math.pi/180)
    
    @staticmethod
    def get_harversion_distance(x1, y1, x2, y2, round_decimal_digits=5):
        """
        경위도 (x1,y1)과 (x2,y2) 점의 거리를 반환
        Harversion Formula 이용하여 2개의 경위도간 거래를 구함(단위:Km)
        """
        if x1 is None or y1 is None or x2 is None or y2 is None:
            return None
        assert isinstance(x1, numbers.Number) and -180 <= x1 and x1 <= 180
        assert isinstance(y1, numbers.Number) and  -90 <= y1 and y1 <=  90
        assert isinstance(x2, numbers.Number) and -180 <= x2 and x2 <= 180
        assert isinstance(y2, numbers.Number) and  -90 <= y2 and y2 <=  90

        R = 6371 # 지구의 반경(단위: km)
        dLon = GeoUtil.degree2radius(x2-x1)    
        dLat = GeoUtil.degree2radius(y2-y1)

        a = math.sin(dLat/2) * math.sin(dLat/2) \
            + (math.cos(GeoUtil.degree2radius(y1)) \
              *math.cos(GeoUtil.degree2radius(y2)) \
              *math.sin(dLon/2) * math.sin(dLon/2))
        b = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
        return round(R * b, round_decimal_digits)

    @staticmethod
    def get_euclidean_distance(x1, y1, x2, y2, round_decimal_digits=5):        
        """
        유클리안 Formula 이용하여 (x1,y1)과 (x2,y2) 점의 거리를 반환
        """
        if x1 is None or y1 is None or x2 is None or y2 is None:
            return None
        assert isinstance(x1, numbers.Number) and -180 <= x1 and x1 <= 180
        assert isinstance(y1, numbers.Number) and  -90 <= y1 and y1 <=  90
        assert isinstance(x2, numbers.Number) and -180 <= x2 and x2 <= 180
        assert isinstance(y2, numbers.Number) and  -90 <= y2 and y2 <=  90

        dLon = abs(x2-x1) # 경도 차이
        if dLon >= 180:   # 반대편으로 갈 수 있는 경우
            dLon -= 360   # 반대편 각을 구한다
        dLat = y2-y1      # 위도 차이
        return round(math.sqrt(pow(dLon,2)+pow(dLat,2)),round_decimal_digits)

In [75]:
center = [36.34722222, 127.39027778] # [위도, 경도] [lat, lon]

lefttop = [36.49638889, 127.24833333]
leftbot = [36.18055556, 127.24833333]
righttop = [36.49638889, 127.55583333]
rightbot = [36.18055556, 127.55583333]

In [64]:
# 세로 높이
print("세로:", GeoUtil.get_harversion_distance(lefttop[1], lefttop[0], leftbot[1], leftbot[0]))
# 가로 너비
print("가로:", GeoUtil.get_harversion_distance(lefttop[1], lefttop[0], righttop[1], righttop[0]))

세로: 35.11906
가로: 27.4871


In [80]:
# 세로 조지기: 35.11906km
height_node = np.arange(leftbot[0],lefttop[0]+0.001, (lefttop[0] - leftbot[0])/70)

In [83]:
# 가로 조지기: 27.4871km
width_node = np.arange(lefttop[1], righttop[1], (righttop[1] - lefttop[1])/55)

In [101]:
len(width_node) * len(height_node)

3976

그리드에 필요한 node list를 만들어줌

In [107]:
node_list = []
for height in height_node:
    for width in width_node:
        node_list.append([height, width])

In [108]:
len(node_list)

3976

In [109]:
node_list[:5]

[[36.18055556, 127.24833333],
 [36.18055556, 127.2539242390909],
 [36.18055556, 127.25951514818182],
 [36.18055556, 127.26510605727273],
 [36.18055556, 127.27069696636364]]

지도에 그리드 놓기

In [118]:
map_daejeon = folium.Map(location=[36.34722222, 127.39027778], zoom_start=14.4)

f = folium.map.FeatureGroup()

for lat, lon in node_list:
    circle = folium.CircleMarker([lat, lon], radius=1, color="#991a1a", fill_color=None)
    
    f.add_child(circle)

In [119]:
map_daejeon.add_child(f)

In [120]:
map_daejeon.save('대전시 on 그리드.html')